# For Colab T4

In [1]:
!pip install datasets
!pip install accelerate
!pip install peft

In [2]:
from datasets import load_dataset, DatasetDict
raw_dataset = load_dataset("nlpai-lab/kullm-v2", split="train")

In [3]:
raw_dataset

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 152630
})

In [4]:
raw_dataset['output'][:200]

['세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.',
 'AI 어시스턴트인 저는 저만의 개인적인 경험이나 물리적 존재를 가지고 있지 않습니다. 하지만 알고리즘, 논리, 데이터를 기반으로 의사 결정을 내리도록 프로그래밍되어 있습니다. 상충되는 정보나 여러 가지 가능한 결과가 있을 수 있기 때문에 의사결정이 어려운 상황에 직면할 때가 있습니다. 그런 경우 저는 각 옵션의 위험과 이점을 분석하고 비교하여 원하는 결과를 얻을 가능성이 가장 높은 결론에 도달할 수 있는 능력을 활용합니다.',
 '원자는 모든 물질의 기본 구성 요소이며 양성자, 중성자, 전자의 세 가지 유형의 입자로 이루어져 있습니다. 원자의 구조는 전자 구름으로 둘러싸인 중앙에 핵이 있는 것으로 설명할 수 있습니다.\n\n원자의 핵은 양성자와 중성자로 구성됩니다. 양성자는 양전하를 띠는 입자이고 중성자는 전하를 띠지 않는 중성 입자입니다. 이 두 입자는 원자의 중심에 있으며 원자 질량의 대부분을 차지하는 원자핵에 위치합니다.\n\n원자핵을 둘러싸고 있는 것은 전자 구름입니다. 전자는 음전하를 띠는 입자로, 원자핵 주변에서 끊임없이 움직입니다. 전자 구름은 껍질 또는 궤도로 나뉘며, 각 껍질은 특정 수의 전자를 보유할 수 있습니다. 원자가 껍질이라고 하는 가장 바깥쪽 껍질에 있는 전자의 수에 따라 원자의 화학적 특성이 결정됩니다.\n\n중성 원자에서 핵의 양성자 수는 전자 구름의 전자 수와 같으므로 양전하와 음전하가 균형을 이루며 원자는 전체 전하를 갖지 않습니다. 원자 번호라고도 하는 양성자의 수는 원자가 어떤 원소인지 결정합니다.',
 '대기 오염을 줄이는 방법에는 다음과 같은 여러 가지가 있습니다:\n\n1. 에너지 소비를 줄입니다: 에너지를 절약하면 발전소에서 배출되는 오염 물질의 양을 줄일 수

In [5]:
sampled_dataset = raw_dataset.select(range(50000))

In [6]:
sampled_dataset

Dataset({
    features: ['id', 'instruction', 'input', 'output'],
    num_rows: 50000
})

## tokenizer

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('psymon/KoLlama2-7b')
tokenizer

c:\Users\mkh08\OneDrive\문서\GitHub\Ollama_TestCode\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


LlamaTokenizerFast(name_or_path='psymon/KoLlama2-7b', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
def get_training_corpus(ds):
    return(
        ds[i:i+1000]['output'] for i in range(0, len(ds), 1000)
    )
training_corpus = get_training_corpus(raw_dataset)

In [9]:
sample_text = "안녕하세요"

tokenizer.tokenize(sample_text)

['▁', '안', '<0xEB>', '<0x85>', '<0x95>', '하', '세', '요']

In [10]:
tokenizer(sample_text, return_length=True)

{'input_ids': [1, 29871, 31734, 238, 136, 152, 30944, 31578, 31527], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1], 'length': [9]}

In [11]:
context_length = 128

def tokenize(batch):
    outputs = tokenizer(
        batch['output'],
        max_length=context_length,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True
    )

    input_batch = []
    for length, input_ids in zip(outputs['length'], outputs['input_ids']):
        if length==context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}

In [12]:
tokenized_datasets = sampled_dataset.map(tokenize, batched=True, remove_columns=raw_dataset.column_names)

In [13]:
sampled_dataset[0]

{'id': 'alpaca_{idx}',
 'instruction': '3원색이란 무엇인가요?',
 'input': '',
 'output': '세 가지 기본 색은 빨강, 파랑, 노랑입니다. 이 색은 다른 색을 혼합하여 만들 수 없고 다른 모든 색은 다양한 비율로 조합하여 만들 수 있기 때문에 원색이라고 부릅니다. 빛에 사용되는 첨가제 색상 시스템에서 원색은 빨강, 녹색, 파랑(RGB)입니다.'}

## load_model

In [14]:
from transformers import LlamaConfig
configuration = LlamaConfig()
configuration

LlamaConfig {
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "transformers_version": "4.38.0",
  "use_cache": true,
  "vocab_size": 32000
}

In [15]:
tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.vocab_size

(1, 2, 32000)

In [16]:
configuration = LlamaConfig(**{
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 1376,
  "max_position_embeddings": 128,
  "model_type": "llama",
  "num_attention_heads": 4,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "rms_norm_eps": 1e-06,
  "tie_word_embeddings": False,
  "transformers_version": "4.28.0",
  "use_cache": True,
  "vocab_size": 50257
})
#0.25B



In [17]:
from transformers import LlamaForCausalLM

model = LlamaForCausalLM(configuration)
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): L

In [18]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
model.to(device)
0

0

In [32]:
prompt = "알고리즘 분석은 고객에게"

inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=50)
generate_ids

tensor([[    1, 29871,   239,   152,   143, 31137, 30826,   239,   169,   155,
         29871,   238,   185,   135,   239,   135,   160, 31354, 29871, 31137,
           237,   179,   160, 31054,   237,   181,   143, 29871,   239,   163,
           132,   239,   160,   148, 30944, 31137, 29871,   239,   161,   139,
           239,   141,   184, 31063, 30709, 29889, 29871, 30393,   238,   162]],
       device='cuda:0')

In [ ]:
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

## train model

In [22]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [23]:
out = data_collator([tokenized_datasets[i] for i in range(3)])

for key in out:
    print(f"{key}: {out[key].shape}")

input_ids: torch.Size([3, 128])
attention_mask: torch.Size([3, 128])
labels: torch.Size([3, 128])


In [24]:
out['input_ids'][0][:20], out['attention_mask'][0][:20], out['labels'][0][:20]

(tensor([    1, 29871, 31578, 29871, 30903, 30811, 29871, 30827,   238,   182,
           187, 29871,   239,   134,   140, 31354, 29871,   238,   188,   171]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 tensor([    1, 29871, 31578, 29871, 30903, 30811, 29871, 30827,   238,   182,
           187, 29871,   239,   134,   140, 31354, 29871,   238,   188,   171]))

In [25]:
from transformers import TrainingArguments

batch_size = 32
logging_steps = 100
learning_rate=3e-3
# learning_rate=5e-4
num_epochs=1

args = TrainingArguments(
    output_dir='testllama',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=logging_steps,
    save_steps=logging_steps,
    gradient_accumulation_steps=8,
    num_train_epochs=num_epochs,
    weight_decay=0.1,
    warmup_steps=logging_steps,
    lr_scheduler_type='cosine',
    learning_rate=learning_rate,
    fp16=True,
    push_to_hub=False
)



In [26]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

In [27]:
trainer.train()

  0%|          | 0/660 [00:00<?, ?it/s]

{'loss': 4.0778, 'grad_norm': 0.5614767074584961, 'learning_rate': 0.003, 'epoch': 0.15}
{'loss': 1.9856, 'grad_norm': 0.26063272356987, 'learning_rate': 0.0027700862988424262, 'epoch': 0.3}
{'loss': 1.6865, 'grad_norm': 0.23928052186965942, 'learning_rate': 0.002150825608676337, 'epoch': 0.45}
{'loss': 1.5701, 'grad_norm': 0.21277426183223724, 'learning_rate': 0.0013320532858450383, 'epoch': 0.61}
{'loss': 1.4812, 'grad_norm': 0.1996116191148758, 'learning_rate': 0.0005647652972118998, 'epoch': 0.76}
{'loss': 1.4221, 'grad_norm': 0.18188048899173737, 'learning_rate': 8.417500453744864e-05, 'epoch': 0.91}
{'train_runtime': 347.5709, 'train_samples_per_second': 486.355, 'train_steps_per_second': 1.899, 'train_loss': 1.9794911124489525, 'epoch': 1.0}


TrainOutput(global_step=660, training_loss=1.9794911124489525, metrics={'train_runtime': 347.5709, 'train_samples_per_second': 486.355, 'train_steps_per_second': 1.899, 'train_loss': 1.9794911124489525, 'epoch': 1.0})

In [31]:
prompt = """알고리즘 분석은 고객에게"""


inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=128)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'알고리즘 분석은 고객에게 적응하고 있습니다. 이러한 작업을 수행하고 있으며, 이는 새로운 생활 습관을 추적하고 있습니다. 이러'

In [34]:
model.save_pretrained('pre_llama')
tokenizer.save_pretrained('pre_llama')

('pre_llama\\tokenizer_config.json',
 'pre_llama\\special_tokens_map.json',
 'pre_llama\\tokenizer.json')

# Instruction Tuning


In [35]:
from transformers import LlamaForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("pre_llama")
model = LlamaForCausalLM.from_pretrained('pre_llama')

model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): L

In [36]:
raw_dataset[6]

{'id': 'alpaca_{idx}',
 'instruction': '주어진 숫자를 오름차순으로 정렬합니다.',
 'input': '2, 4, 0, 8, 3',
 'output': '0, 2, 3, 4, 8.'}

In [37]:

# def gen_prompt(element):
#     prompt_format = """#Instruction: %s \n %s \n #result: \n %s"""
#     return DatasetDict({'input': prompt_format%(element['instruction'], element['input'],  element['output'])})

def gen_prompt(element):
    instruction = element['instruction']
    input_text = element['input']
    output_text = element['output']

    # 줄바꿈 문자를 포함하여 문자열 포맷팅
    prompt_format = f"지시: {instruction} \n {input_text}\n 결과: {output_text}"

    return DatasetDict({'input': prompt_format})


dataset = sampled_dataset.map(gen_prompt)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [38]:
dataset[6]

{'id': 'alpaca_{idx}',
 'instruction': '주어진 숫자를 오름차순으로 정렬합니다.',
 'input': '지시: 주어진 숫자를 오름차순으로 정렬합니다. \n 2, 4, 0, 8, 3\n 결과: 0, 2, 3, 4, 8.',
 'output': '0, 2, 3, 4, 8.'}

In [39]:
def tokenize(element):
    tokenizer.pad_token = tokenizer.eos_token
    outputs = tokenizer(
        element['input'],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=False,
        return_length=True,
        padding=True
    )
    input_batch = []
    for input, input_ids, output in zip(element["input"], outputs["input_ids"], element['output']):
        input_batch.append(input_ids)
    return {"input_ids": input_batch}

context_length=128
tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=raw_dataset.column_names
)
tokenized_datasets

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids'],
    num_rows: 50000
})

In [40]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [41]:
out = data_collator([tokenized_datasets[i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [42]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="test_llama_inst",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=1000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=1000,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

In [43]:
trainer.train()

  0%|          | 0/1562 [00:00<?, ?it/s]

{'loss': 1.2987, 'grad_norm': 0.5819985270500183, 'learning_rate': 0.0005, 'epoch': 0.64}
{'train_runtime': 201.6207, 'train_samples_per_second': 247.99, 'train_steps_per_second': 7.747, 'train_loss': 1.265442324356294, 'epoch': 1.0}


TrainOutput(global_step=1562, training_loss=1.265442324356294, metrics={'train_runtime': 201.6207, 'train_samples_per_second': 247.99, 'train_steps_per_second': 7.747, 'train_loss': 1.265442324356294, 'epoch': 1.0})

In [45]:
question = "주어진 숫자를 오름차순으로 정렬합니다. \n 2, 4, 0, 8, 3\n "
prompt = f"""#지시:{question}
#결과: """

inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=128)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'#지시:주어진 숫자를 오름차순으로 정렬합니다. \n 2, 4, 0, 8, 3\n \n#결과: 숫자 1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10'

In [47]:
question = "머신러닝을 사용하여 일상적인 작업을 자동화하는 방법을 설명합니다."
prompt = f"""#지시:{question}
#결과: """


inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

generate_ids = model.generate(inputs.input_ids, max_length=128)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'#지시:머신러닝을 사용하여 일상적인 작업을 자동화하는 방법을 설명합니다.\n#결과: 머신 러닝 알고리즘은 머신 러닝 알고리즘을 사용하여 생산성을 높이는 �'

In [48]:
model.save_pretrained('inst_llama')
tokenizer.save_pretrained('inst_llama')

('inst_llama\\tokenizer_config.json',
 'inst_llama\\special_tokens_map.json',
 'inst_llama\\tokenizer.json')

# PeFT

In [49]:
!pip install peft

In [50]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [51]:
from transformers import LlamaForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('inst_llama')
model = LlamaForCausalLM.from_pretrained('inst_llama')

model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(50257, 512, padding_idx=0)
    (layers): ModuleList(
      (0-3): 4 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=512, out_features=512, bias=False)
          (k_proj): Linear(in_features=512, out_features=512, bias=False)
          (v_proj): Linear(in_features=512, out_features=512, bias=False)
          (o_proj): Linear(in_features=512, out_features=512, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=512, out_features=1376, bias=False)
          (up_proj): Linear(in_features=512, out_features=1376, bias=False)
          (down_proj): Linear(in_features=1376, out_features=512, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): L

In [53]:
from peft import get_peft_model, LoraConfig, TaskType
list(TaskType)

[<TaskType.SEQ_CLS: 'SEQ_CLS'>,
 <TaskType.SEQ_2_SEQ_LM: 'SEQ_2_SEQ_LM'>,
 <TaskType.CAUSAL_LM: 'CAUSAL_LM'>,
 <TaskType.TOKEN_CLS: 'TOKEN_CLS'>,
 <TaskType.QUESTION_ANS: 'QUESTION_ANS'>,
 <TaskType.FEATURE_EXTRACTION: 'FEATURE_EXTRACTION'>]

In [54]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
                        inference_mode=False,
                        r=16,
                        lora_alpha=16, 
                        lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.to(device)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(50257, 512, padding_idx=0)
        (layers): ModuleList(
          (0-3): 4 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=512, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=512, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=512, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=512, out_features=512, bias=False)
 

In [55]:
model.print_trainable_parameters()

trainable params: 262,144 || all params: 64,378,368 || trainable%: 0.40719267689420147


In [56]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

out = data_collator([tokenized_datasets[i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [57]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="peft_llama",
    per_device_train_batch_size=4,
    logging_steps=500,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=500,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=1000,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

In [58]:
trainer.train()

  0%|          | 0/1562 [00:00<?, ?it/s]

{'loss': 1.144, 'grad_norm': 0.026584060862660408, 'learning_rate': 0.0005, 'epoch': 0.32}
{'loss': 1.1116, 'grad_norm': 0.0339759886264801, 'learning_rate': 0.00027289381569935167, 'epoch': 0.64}
{'loss': 1.0791, 'grad_norm': 0.034287773072719574, 'learning_rate': 4.193014378207044e-06, 'epoch': 0.96}
{'train_runtime': 191.5604, 'train_samples_per_second': 261.014, 'train_steps_per_second': 8.154, 'train_loss': 1.1130574430302071, 'epoch': 1.0}


TrainOutput(global_step=1562, training_loss=1.1130574430302071, metrics={'train_runtime': 191.5604, 'train_samples_per_second': 261.014, 'train_steps_per_second': 8.154, 'train_loss': 1.1130574430302071, 'epoch': 1.0})

In [59]:
model.save_pretrained('peft_llama_adapter')

# PEFT Model LOAD

In [60]:
from transformers import LlamaForCausalLM
from peft import PeftModel, PeftConfig
base_model =  LlamaForCausalLM.from_pretrained('inst_llama')
model_load = PeftModel.from_pretrained(base_model, 'peft_llama_adapter')
model_load.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(50257, 512, padding_idx=0)
        (layers): ModuleList(
          (0-3): 4 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=512, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=512, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=512, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=512, out_features=512, bias=False)
 

In [61]:
model = model_load.merge_and_unload()

In [62]:
import os
os.stat('peft_llama_adapter/adapter_model.safetensors').st_size/(1024*1024)

1.0020065307617188

In [63]:
os.stat("inst_llama/model.safetensors").st_size/(1024*1024)

244.58809661865234

In [67]:
question = "주어진 숫자를 오름차순으로 정렬합니다. \n 2, 4, 0, 8, 3\n "
prompt = f"""#지시:{question}
#결과: """

inputs = tokenizer(prompt, return_tensors='pt')
inputs.to(device)

{'input_ids': tensor([[    1,   396, 30811, 30889, 29901, 30981, 31129, 31536, 29871,   239,
           139,   174, 31013, 31517, 29871, 31346,   238,   169,   135, 31817,
           239,   139,   159,   239,   159,   191, 30906, 29871, 30852,   238,
           163,   175, 31980, 31063, 30709, 29889, 29871,    13, 29871, 29906,
         29892, 29871, 29946, 29892, 29871, 29900, 29892, 29871, 29947, 29892,
         29871, 29941,    13, 29871,    13, 29937,   237,   181,   179, 31906,
         29901, 29871]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [68]:
generate_ids = model.generate(inputs.input_ids, max_length=128)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'#지시:주어진 숫자를 오름차순으로 정렬합니다. \n 2, 4, 0, 8, 3\n \n#결과: 숫자 1, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10'